### Ichimoku Kinko Hyo
a fairly complete indicator, 3 temporal variables and 5 curves. momentum, supports, resistances, and strength of signals

curve 1 - chikou span
closing value of share shifted 26 periods backwards. Lagging span, filter to confirm other signals

curve 2 - kijun sen
average level of values over the medium-term time interval (26 sessions)
medium term trend evaluator - the BASELINE
highest value in period plus lowest value in period, divided by 2

curve 3 - tenkan sen
average levels of values over short term time interval (9 session)
short term trend evaluator - the CONVERSION LINE
same formula as above

curve 4 - senkou span A
first boundary of the kumo cloud. average distance between the tenkan sen line and the kijun sen line, shifted over 26 sessions.
tenkan sen plus kijun sen, divided by 2

curve 5 - senkou span B
second boundary of the kumo cloud. average distance between tenkan sen and kijun sen, over 52 sessions.
long term bias makes this a support or resistance line depending on its spot. shifted over 26 sessions.
same formula as above

Kumo cloud - area between senkou span A and senkou span B. Represents divergence in price evolution.

In [7]:
import pandas as pd
from pandas_datareader import data, wb
import matplotlib.dates as dates
import datetime

start = pd.to_datetime('2018-02-04')
end = pd.to_datetime('2020-05-29')

In [8]:
df = data.DataReader('TSLA', 'yahoo', start, end)

In [9]:
# ICHIMOKU implementation

# Tenkan-sen (Conversion Line) : (9 period high + 9 period low) / 2
df['tenkan_sen'] = (df['High'].rolling(window=9).max() + df['Low'].rolling(window=9).min()) / 2

# Kijun-sen (Base Line) : (26 period high + 26 period low) / 2
df['kijun_sen'] = (df['High'].rolling(window=26).max() + df['Low'].rolling(window=26).min()) / 2

# Senkou Span A : (Conversion Line + Base Line) / 2
df['senkou_span_a'] = ((df['tenkan_sen'] + df['kijun_sen']) / 2).shift(26)

# Senkou Span B : (52 period high + 52 period low) / 2
df['senkou_span_b'] = ((df['High'].rolling(window=52).max() + df['Low'].rolling(window=52).min()) / 2).shift(26)

# Chikou Span (Optional) : 26 period lagging closing price
df['chikou_span'] = df['Close'].shift(-26)

In [10]:
df

,High,Low,Open,Close,Volume,Adj Close,tenkan_sen,kijun_sen,senkou_span_a,senkou_span_b,chikou_span
Date,,,,,,,,,,,
2018-02-05,68.893997,66.599998,67.594002,66.625999,22320500,66.625999,NaN,NaN,NaN,NaN,65.325996
2018-02-06,67.244003,64.699997,65.042000,66.793999,25442000,66.793999,NaN,NaN,NaN,NaN,65.120003
2018-02-07,69.199997,67.132004,67.797997,69.000000,34846000,69.000000,NaN,NaN,NaN,NaN,64.269997
2018-02-08,69.723999,62.919998,68.662003,63.046001,51573000,63.046001,NaN,NaN,NaN,NaN,62.712002
2018-02-09,64.195999,58.952000,63.986000,62.084000,64668500,62.084000,NaN,NaN,NaN,NaN,62.110001
...,...,...,...,...,...,...,...,...,...,...,...
2020-05-22,166.356003,162.399994,164.434006,163.376007,9976900,163.376007,160.659004,154.361000,116.889999,131.950001,NaN
2020-05-26,166.919998,163.141998,166.899994,163.774002,40448500,163.774002,159.802002,154.361000,119.918503,131.950001,NaN
2020-05-27,165.542007,157.000000,164.171997,164.046005,57747500,164.046005,159.872002,154.361000,121.637503,129.528996,NaN


In [12]:
# Visualize the curves
import plotly.graph_objs as go
fig = go.Figure()

fig.add_trace(go.Scatter(x=df.index, y=df['tenkan_sen'], line=dict(color='royalblue', width=.8), name='Tenkan Sen'))
fig.add_trace(go.Scatter(x=df.index, y=df['kijun_sen'], line=dict(color='orange', width=.8), name='Kijun Sen'))
fig.add_trace(go.Scatter(x=df.index, y=df['senkou_span_a'], line=dict(color='black', width=.8), name='Senkou Span A'))
fig.add_trace(go.Scatter(x=df.index, y=df['senkou_span_b'], line=dict(color='purple', width=1.8), name='Senkou Span B'))
fig.add_trace(go.Scatter(x=df.index, y=df['chikou_span'], line=dict(color='red', width=.8), name='Chinkou Span'))

fig.add_trace(go.Candlestick(x=df.index,
                            open=df['Open'],
                            high=df['High'],
                            low=df['Low'],
                            close=df['Close'],
                            name='Market Data (USD)'))

fig.update_layout(
    title='Ichimoku Strategy',
    yaxis_title='Tesla Stock (USD Per Share)')

fig.show()